# Sensitivity Analysis for Insurance Optimization

## Overview
Systematic sensitivity analysis to understand how parameter changes affect optimal insurance decisions and business outcomes.  Covers one-at-a-time sweeps, tornado diagrams, two-way interaction heatmaps, and market-scenario stress testing.

- **Prerequisites**: [optimization/01_optimization_overview](01_optimization_overview.ipynb)
- **Estimated runtime**: 2-4 minutes
- **Audience**: [Practitioner]

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

from ergodic_insurance.sensitivity import (
    SensitivityAnalyzer,
    SensitivityResult,
    TwoWaySensitivityResult,
)
from ergodic_insurance.sensitivity_visualization import (
    plot_tornado_diagram,
    plot_two_way_sensitivity,
    plot_parameter_sweep,
    plot_sensitivity_matrix,
    create_sensitivity_report,
)
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.business_optimizer import (
    BusinessOptimizer, BusinessConstraints, BusinessObjective,
)

plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")
%matplotlib inline

# Reproducibility
SEED = 42
np.random.seed(SEED)

## 1. Base Configuration

The `SensitivityAnalyzer` takes a baseline configuration dictionary and an optimizer that maps any configuration to optimization results.

In [ ]:
base_config = {
    # Manufacturing parameters
    "initial_assets": 10_000_000,
    "asset_turnover": 1.0,
    "base_operating_margin": 0.08,
    "tax_rate": 0.25,
    # Loss parameters
    "loss_frequency": 5.0,
    "loss_severity_mean": 100_000,
    "loss_severity_cv": 1.5,
    # Insurance parameters
    "base_premium_rate": 0.02,
    "deductible": 50_000,
    "coverage_limit": 5_000_000,
    # Constraints
    "max_ruin_probability": 0.01,
    "min_roe_target": 0.15,
    "max_premium_budget": 0.03,
}

print("Base Configuration:")
for k, v in base_config.items():
    if isinstance(v, float) and v < 1:
        print(f"  {k}: {v:.1%}")
    elif isinstance(v, (int, float)) and v > 1000:
        print(f"  {k}: ${v:,.0f}")
    else:
        print(f"  {k}: {v}")

In [ ]:
# Lightweight optimizer for demonstration (deterministic model)
from dataclasses import dataclass

@dataclass
class _Strategy:
    expected_roe: float
    bankruptcy_risk: float
    growth_rate: float
    capital_efficiency: float
    deductible: float
    premium_rate: float

class DemoOptimizer:
    """Simplified optimizer that returns deterministic results from config values."""
    def optimize(self, config):
        freq = config.get("loss_frequency", 5)
        sev = config.get("loss_severity_mean", 100_000)
        prem = config.get("base_premium_rate", 0.02)
        ded = config.get("deductible", 50_000)
        margin = config.get("base_operating_margin", 0.08)

        base_roe = margin * 2
        freq_impact = (5 - freq) * 0.01
        sev_impact = (100_000 - sev) / 1_000_000
        prem_impact = -prem * 2

        class _Result:
            optimal_strategy = _Strategy(
                expected_roe=max(0.05, min(0.30, base_roe + freq_impact + sev_impact + prem_impact)),
                bankruptcy_risk=max(0.001, min(0.05, freq * sev / 100_000_000)),
                growth_rate=max(0.02, min(0.15, margin * 0.8)),
                capital_efficiency=0.75 + margin,
                deductible=ded,
                premium_rate=prem,
            )
        return _Result()

optimizer = DemoOptimizer()

## 2. Initialize Sensitivity Analyzer

In [ ]:
cache_dir = Path("cache/sensitivity")
cache_dir.mkdir(parents=True, exist_ok=True)

analyzer = SensitivityAnalyzer(
    base_config=base_config,
    optimizer=optimizer,
    cache_dir=cache_dir,
)
print(f"Analyzer ready  |  {len(base_config)} parameters  |  cache: {cache_dir}")

## 3. One-at-a-Time (OAT) Parameter Sweep

Vary each parameter individually while holding others at baseline.

In [ ]:
freq_result = analyzer.analyze_parameter(
    "loss_frequency", param_range=(3, 8), n_points=11,
)

print(f"Loss Frequency sweep: {freq_result.variations[0]:.1f} - {freq_result.variations[-1]:.1f}")
for metric in ["optimal_roe", "bankruptcy_risk", "growth_rate"]:
    impact = freq_result.calculate_impact(metric)
    lo, hi = freq_result.get_metric_bounds(metric)
    print(f"  {metric:20s}  elasticity={impact:+.3f}  range=[{lo:.3f}, {hi:.3f}]")

In [ ]:
fig = plot_parameter_sweep(
    freq_result,
    metrics=["optimal_roe", "bankruptcy_risk", "growth_rate"],
    title="Impact of Loss Frequency on Key Metrics",
    figsize=(12, 4),
    mark_baseline=True,
)
plt.tight_layout()
plt.show()

## 4. Tornado Diagram -- Parameter Impact Ranking

In [ ]:
key_parameters = [
    "loss_frequency",
    "loss_severity_mean",
    "loss_severity_cv",
    "base_premium_rate",
    "deductible",
    "base_operating_margin",
    "asset_turnover",
]

tornado_data = analyzer.create_tornado_diagram(
    parameters=key_parameters,
    metric="optimal_roe",
    relative_range=0.3,
    n_points=5,
)

print("Parameter Impact Ranking (by ROE impact range):")
for i, row in tornado_data.iterrows():
    print(f"  {i+1}. {row['parameter']:25s}  impact={row['impact']:.3f}")

In [ ]:
fig = plot_tornado_diagram(
    tornado_data,
    title="Sensitivity Tornado Diagram",
    metric_label="Impact on Expected ROE",
    figsize=(10, 6),
    n_params=7,
    show_values=True,
)
plt.tight_layout()
plt.show()

## 5. Two-Way Sensitivity (Interaction Heatmap)

Explore how **loss frequency** and **loss severity** interact.

In [ ]:
two_way = analyzer.analyze_two_way(
    param1="loss_frequency",
    param2="loss_severity_mean",
    param1_range=(3, 8),
    param2_range=(50_000, 200_000),
    n_points1=8,
    n_points2=8,
    metric="optimal_roe",
)

print(f"ROE grid: {two_way.metric_grid.shape}  "
      f"range=[{two_way.metric_grid.min():.3f}, {two_way.metric_grid.max():.3f}]")

In [ ]:
fig = plot_two_way_sensitivity(
    two_way,
    title="ROE Sensitivity: Loss Frequency vs Severity",
    cmap="RdYlGn",
    figsize=(10, 8),
    show_contours=True,
    contour_levels=10,
    optimal_point=(5.0, 100_000),
    fmt=".2%",
)
plt.tight_layout()
plt.show()

In [ ]:
# Feasibility region: where ROE > 15%
TARGET_ROE = 0.15
optimal_mask = two_way.find_optimal_region(TARGET_ROE, tolerance=0.1)
n_feasible = optimal_mask.sum()
n_total = optimal_mask.size

print(f"Target ROE: {TARGET_ROE:.0%}")
print(f"Feasible cells: {n_feasible}/{n_total} ({100 * n_feasible / n_total:.0f}%)")

fig, ax = plt.subplots(figsize=(10, 8))
X, Y = np.meshgrid(two_way.values1, two_way.values2, indexing="ij")
ax.contourf(X, Y, optimal_mask.astype(float), levels=[0, 0.5, 1],
            colors=["#ffcccc", "#ccffcc"], alpha=0.4)
ax.contour(X, Y, two_way.metric_grid, levels=[TARGET_ROE],
           colors="black", linewidths=2)
ax.set_xlabel("Loss Frequency (claims/year)")
ax.set_ylabel("Loss Severity Mean ($)")
ax.set_title(f"Feasible Region for ROE > {TARGET_ROE:.0%}")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Multi-Metric Cross-Parameter Comparison

In [ ]:
metrics_to_compare = ["optimal_roe", "bankruptcy_risk", "growth_rate", "capital_efficiency"]
params_to_compare = ["loss_frequency", "loss_severity_mean",
                     "base_premium_rate", "base_operating_margin"]

impact_df = pd.DataFrame(index=metrics_to_compare, columns=params_to_compare, dtype=float)

for param in params_to_compare:
    result = analyzer.analyze_parameter(param, relative_range=0.3, n_points=5)
    for metric in metrics_to_compare:
        impact_df.loc[metric, param] = abs(result.calculate_impact(metric))

fig, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(impact_df.astype(float), annot=True, fmt=".3f", cmap="YlOrRd",
            cbar_kws={"label": "Elasticity (absolute)"})
ax.set_title("Parameter Sensitivity Across Different Metrics")
plt.tight_layout()
plt.show()

print("Most sensitive parameter per metric:")
for metric in metrics_to_compare:
    best = impact_df.loc[metric].astype(float).idxmax()
    print(f"  {metric:25s} -> {best}")

## Key Takeaways

- **Operating margin** and **loss frequency** are usually the most impactful parameters on ROE.
- **Two-way analysis** reveals interaction effects that single-parameter sweeps miss (e.g., high frequency *and* high severity can push the company out of the feasible region).
- The `SensitivityAnalyzer` caches results automatically, so repeated analyses are fast.

## Next Steps

- [optimization/03_pareto_analysis](03_pareto_analysis.ipynb) -- multi-objective trade-offs on a Pareto frontier
- [optimization/05_parameter_sweeps](05_parameter_sweeps.ipynb) -- grid-search over larger parameter spaces
- [optimization/04_retention_optimization](04_retention_optimization.ipynb) -- detailed deductible optimization